In [12]:
import pandas as pd
import numpy as np

listingsDF = pd.read_csv('listings.csv')

calendarDF = pd.read_csv('calendar.csv')

reviewsDF = pd.read_csv('reviews.csv')

In [13]:
#We clean reviews first

#We need to remove '(email hidden)', special symbols
reviewsDF = reviewsDF.loc[reviewsDF['reviewer_name'].str.contains(r'[^\x00-\x7F]+') == False]

reviewsDF = reviewsDF.loc[reviewsDF['reviewer_name'].str.contains('email hidden') == False]

#Then replace '()', '-'
reviewsDF.reviewer_name = reviewsDF.reviewer_name.str.replace('[()]', '')

reviewsDF.reviewer_name = reviewsDF.reviewer_name.str.replace('[-]', ' ')

#Then replace 'And' with '&'
reviewsDF.reviewer_name = reviewsDF.reviewer_name.str.replace('And', '&')

#Now we remove all rows with empty cells
reviewsDF = reviewsDF.dropna(axis=0, how='any')

#Convert date from object into datetime
reviewsDF.date = pd.to_datetime(reviewsDF.date)

#Check for row
#reviewsDF.loc[reviewsDF['reviewer_id'] == 12120141]

reviewsDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84283 entries, 0 to 84848
Data columns (total 6 columns):
listing_id       84283 non-null int64
id               84283 non-null int64
date             84283 non-null datetime64[ns]
reviewer_id      84283 non-null int64
reviewer_name    84283 non-null object
comments         84283 non-null object
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 4.5+ MB


In [16]:
#Now we clean listings

#Instead of dropping we select what we need
listingsDF = listingsDF[['id', 'name','description', 
                         'host_id', 'host_name', 'property_type', 'price', 
                         'number_of_reviews', 'review_scores_rating']]

#We drop all rows with empty cells
listingsDF = listingsDF.dropna(axis=0, how='any')

#Remove the '$' from price
listingsDF.price = listingsDF.price.str.replace('[$]', '')

#Remove the ',' from price
listingsDF.price = listingsDF.price.str.replace(',', '')

#Replace host name 'And' with '&'
listingsDF.price = listingsDF.price.str.replace('And', '&')

#Convert price from object to float
listingsDF['price'] = listingsDF['price'].astype(float)

#listingsDF.head()
listingsDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3171 entries, 0 to 3814
Data columns (total 9 columns):
id                      3171 non-null int64
name                    3171 non-null object
description             3171 non-null object
host_id                 3171 non-null int64
host_name               3171 non-null object
property_type           3171 non-null object
price                   3171 non-null float64
number_of_reviews       3171 non-null int64
review_scores_rating    3171 non-null float64
dtypes: float64(2), int64(3), object(4)
memory usage: 247.7+ KB


In [18]:
#We want to merge these 2 DFs, as our main focus is on the reviews, we will use reviewsDF as main
#In reviewsDF, "id" is the primary key, "listing_id" is the foreign key
#In listingsDF, "id" is the primary key
#We want to select where reviews.listings_id = listings.id;
#But since the column name is different, we will rename listingsDF.id to listingsDF.listing_id

listingsDF.rename(columns={'id': 'listing_id'}, inplace=True)
reviewsMDF = pd.merge(reviewsDF, listingsDF, on="listing_id")

reviewsMDF.head()
reviewsMDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84263 entries, 0 to 84262
Data columns (total 14 columns):
listing_id              84263 non-null int64
id                      84263 non-null int64
date                    84263 non-null datetime64[ns]
reviewer_id             84263 non-null int64
reviewer_name           84263 non-null object
comments                84263 non-null object
name                    84263 non-null object
description             84263 non-null object
host_id                 84263 non-null int64
host_name               84263 non-null object
property_type           84263 non-null object
price                   84263 non-null float64
number_of_reviews       84263 non-null int64
review_scores_rating    84263 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(5), object(6)
memory usage: 9.6+ MB


In [19]:
#Im gonna export to csv to view the data
reviewsMDF.to_csv('Merged.csv')